In [1]:
!pip install datasets
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import datasets
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.model_selection import StratifiedKFold

In [3]:
dataset = datasets.load_dataset('gnad10')
train_texts = dataset['train']

  0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
tokenizer = BertTokenizer.from_pretrained('bert-base-german-cased')

In [6]:
def tokenize_dataset(data):
    # Keys of the returned dictionary will be added to the dataset as columns
    return tokenizer(data["text"], padding=True, max_length=128, truncation=True)


dataset = train_texts.map(tokenize_dataset)

Map:   0%|          | 0/9245 [00:00<?, ? examples/s]

In [9]:
from keras.optimizers import Adam
import keras
model = TFBertForSequenceClassification.from_pretrained('bert-base-german-cased', num_labels=9)
model.compile(optimizer=Adam(1e-5), metrics='accuracy')

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-german-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [10]:
tf_dataset = model.prepare_tf_dataset(dataset, batch_size=4, shuffle=True, tokenizer=tokenizer)

In [12]:
model.fit(tf_dataset, epochs=2, verbose=1)

Epoch 1/2
2311/2311 [==============================] - 304s 124ms/step - loss: 0.5827 - accuracy: 0.8251
Epoch 2/2
2311/2311 [==============================] - 296s 128ms/step - loss: 0.2452 - accuracy: 0.9217
